<a href="https://colab.research.google.com/github/shahd1995913/classification-the-fake-and-real-news/blob/main/Check_your_medical_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
     
true = pd.read_csv('/content/truedata.csv' , encoding= 'unicode_escape')
true.head(3)

In [ ]:
true.shape

In [ ]:
fake = pd.read_csv('/content/fakedata.csv' , encoding= 'unicode_escape')
fake.shape

In [ ]:
true['label'] = 1
fake['label'] = 0

In [ ]:
frames = [true.loc[:150][:], fake.loc[:150][:]]

In [ ]:
df = pd.concat(frames)


In [ ]:
df.shape


In [ ]:
df.head()

In [ ]:
X = df. drop('label', axis=1)
y = df['label']
     

df = df.dropna()
df2 = df.copy()
     

df2.head()

In [ ]:

df2.reset_index(inplace=True)
df2.head()
     

In [ ]:
df2['title'][2]


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
     

import re
import nltk
nltk.download('stopwords')

corpus = []
for i in range(0, len(df2)):
    review = re.sub('[^a-zA-Z]', ' ', df2['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
     

In [ ]:
# TFidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
     

X = tfidf_v.fit_transform(corpus).toarray()
y = df2['label']
     

# Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
classifier = PassiveAggressiveClassifier(max_iter=1000)
     

from sklearn import metrics
import numpy as np
import itertools

classifier.fit(X_train, y_train)

pred = classifier.predict(X_test)

score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

In [ ]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
     

cm = metrics.confusion_matrix(y_test, pred)
plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])

In [ ]:
review = re.sub('[^a-zA-Z]', ' ', fake['text'][300])
review = review.lower()
review = review.split()
    
review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
review

In [ ]:
val = tfidf_v.transform([review]).toarray()
     
classifier.predict(val)

In [ ]:
import pickle
     

pickle.dump(classifier, open('model2.pkl', 'wb'))
     

pickle.dump(tfidf_v, open('tfidfvect2.pkl', 'wb'))

In [ ]:

joblib_model = pickle.load(open('model2.pkl', 'rb'))
     

joblib_vect = pickle.load(open('tfidfvect2.pkl', 'rb'))
     

val_pkl = joblib_vect.transform([review]).toarray()
     

joblib_model.predict(val_pkl)